# 3.1 Trends 

In [ ]:
# Here, we use the calculation of temperature trends as an example and provide key sample code.

In [ ]:
import numpy as np
from osgeo import gdal
import os
import pandas as pd
import pymannkendall as mk

In [ ]:
def writeTif(tif_data, tif_geotrans, tif_proj, save_path):
    driver = gdal.GetDriverByName("GTiff")
    cols = tif_data.shape[1]  # 矩阵列数
    rows = tif_data.shape[0]  # 矩阵行数
    dataset = driver.Create(save_path, cols, rows, 1, gdal.GDT_Float32)
    dataset.SetGeoTransform(tif_geotrans)

    dataset.SetProjection(tif_proj)
    dataset.GetRasterBand(1).WriteArray(tif_data)
    del dataset

In [ ]:
def readTif(fileName):
    dataset = gdal.Open(fileName)
    if dataset == None:
        print(fileName+"文件无法打开")
    return dataset

In [ ]:
year_range = np.linspace(1982, 2014, 33, dtype = int)

In [ ]:
# Tmin变化趋势

tem_dir = '/data/shuping/data/climate_data/CRU TS V4.01/tif/tmn/'
trend_data = np.full((120,720), np.nan)
trend_p_data = np.full((120,720), np.nan)
trend_pass_test_data = np.full((120,720), np.nan)
for j in range(12):
    tmean_month_datasat = np.zeros((33, 120, 720))
    for i in range(len(year_range)):
        files = sorted(os.listdir(os.path.join(tem_dir, str(year_range[i]))))
        tem_data = readTif(os.path.join(tem_dir, str(year_range[i]), files[j]))
        tem = tem_data.ReadAsArray()
        tmean_month_datasat[i] = tem
    for k in range(120):
        for g in range(720):
            time_series = tmean_month_datasat[:, k, g]
            a = np.isnan(time_series).sum()
            if a == 0:
                trend, h, p, z, Tau, s, var_s, slope, intercept = mk.original_test(time_series)
                trend_data[k, g] = slope
                trend_p_data[k, g] = p
                if trend_p_data[k, g] <= 0.05:
                    trend_pass_test_data[k, g] = slope
    writeTif(trend_data, tif_geotrans, tif_proj, os.path.join('/data/shuping/data/tmin', str(j+1) + '_tmin.tif'))
    writeTif(trend_pass_test_data, tif_geotrans, tif_proj, os.path.join(
             '/data/shuping/data/tmin', str(j+1) + '_tmin_pass_test.tif'))
    print(j)